# December 09, 2021

https://adventofcode.com/2021/day/9

In [1]:
import pandas as pd
import numpy as np

In [2]:
with open("../data/2021/09.txt", "r") as f:
    data = f.read()

In [3]:
test_str = "2199943210\n3987894921\n9856789892\n8767896789\n9899965678"

In [4]:
class TubeMap:
    def __init__( self, data, verbose = False ):
        # construct from a single string with \n to separate lines
        # imagine that... using a new line separator to separate lines!
        lines = data.split("\n")

        # remove blank line if data ends in newline
        if len(lines[-1]) == 0:
            lines = lines[:-1]
        ncols = [len(x) for x in lines]

        if max(ncols) != min(ncols):
            raise "Not all lines have the same length!"
            
        self.nc = ncols[0]
        self.nr = len(lines)
        self.heights = lines
        self.unexplored = [ (i,j) for i in range(self.nr) for j in range(self.nc) ]
        self.basins = []
        self.verbose = verbose
            
    def display( self ):
        print(f'''{self.nr} x {self.nc} map of tube heights''')
        if self.nr * self.nc < 1000:
            display(self.heights)
        else:
            print("It's a biggun!")

    def is_nadir( self, i, j ):
        '''is (i,j) a nadir for this TubeMap?'''

        # check above
        if i > 0 and self.heights[i][j] >= self.heights[i-1][j]:
            return False
        # check below
        if i < self.nr-1 and self.heights[i][j] >= self.heights[i+1][j]:
            return False
        # check left
        if j > 0 and self.heights[i][j] >= self.heights[i][j-1]:
            return False
        # check right
        if j < self.nc-1 and self.heights[i][j] >= self.heights[i][j+1]:
            return False
        return True
        
    def risk( self ):
        '''return total risk for this TubeMap'''
        risk = 0
        for i in range(self.nr):
            for j in range(self.nc):
                risk += self.is_nadir(i,j) * (1 + int(self.heights[i][j]) )

        return risk
    
    def map_basin( self, i, j ):
        '''map or return a basin that includes (i,j)'''
        # see if basin is already known
        if not self._in_basin(i,j):
            try:
                self.unexplored.remove( (i,j) )
            except:
                pass
            return None
        
        for basin in self.basins:
            if (i,j) in basin: return basin

        # otherwise map the new basin
        new_basin = self._map_basin( i, j )
        self.basins.append( new_basin )
        self._sort_basins() # maintain basins in sorted order

        return new_basin
        
    def map_all( self ):
        '''map all the basins'''
        while len( self.unexplored ) > 0:
            self.map_basin( self.unexplored[0][0], self.unexplored[0][1] )

    def get_basins( self ):
        # get basins by size large to small
        self.map_all()
        return self.basins
    
    def get_biggest_basins( self, n=1 ):
        self.get_basins()
        return self.basins[ :min(n, len(self.basins)) ]
    
    def _in_basin( self, i, j ):
        '''is (i,j) a basin?'''
        # according to the problem statement, everything by 9 is in a basin
        # however, this conflicts with the definition of basin which makes it seem
        # like any local max is not in a basin

        return self.heights[i][j] < "9"
    
    def _map_basin( self, i, j ):
        '''supports map_basin. Do not call directly'''
        
        # if (i,j) is already explored (or an illegal coordinate!) there's nothing to explore.
        try:
            self.unexplored.remove( (i,j) )
        except:
            return []
        
        # if (i,j) is not in a basin, do not add it to the one we're exploring
        if not self._in_basin( i, j ): return []

        # otherwise, return (i,j) and neighboring points in the basin
        if self.verbose: print(i,j)

        return (
            [ (i,j) ]
            + self._map_basin( i-1, j )
            + self._map_basin( i+1, j )
            + self._map_basin( i, j-1 )
            + self._map_basin( i, j+1 )
        )
    
    def _sort_basins(self):
        self.basins.sort( reverse = True, key = lambda x: len(x) )

# Part 1

In [5]:
tm = TubeMap(test_str)
tm.display()

5 x 10 map of tube heights


['2199943210', '3987894921', '9856789892', '8767896789', '9899965678']

In [6]:
tm.risk()

15

In [7]:
tm = TubeMap(data)
tm.display()

100 x 100 map of tube heights
It's a biggun!


In [8]:
tm.risk()

633

# Part 2

In [9]:
tm = TubeMap(test_str, verbose = True)
tm.display()

5 x 10 map of tube heights


['2199943210', '3987894921', '9856789892', '8767896789', '9899965678']

In [10]:
# first time we have to map the basin
print( tm.map_basin(0,0) )

# second time we do the lookup
print( tm.map_basin(0,0) )

# another point in the basin can also be looked up
print( tm.map_basin(1,0) )

0 0
1 0
0 1
[(0, 0), (1, 0), (0, 1)]
[(0, 0), (1, 0), (0, 1)]
[(0, 0), (1, 0), (0, 1)]


In [11]:
tm.map_basin(0,9)

0 9
1 9
2 9
1 8
0 8
0 7
0 6
1 6
0 5


[(0, 9), (1, 9), (2, 9), (1, 8), (0, 8), (0, 7), (0, 6), (1, 6), (0, 5)]

In [12]:
# will spam for any new basins
tm.get_basins()

1 2
2 2
3 2
3 1
2 1
4 1
3 0
3 3
2 3
1 3
1 4
2 4
3 4
2 5
2 7
3 7
4 7
4 6
3 6
4 5
4 8
3 8
4 9


[[(1, 2),
  (2, 2),
  (3, 2),
  (3, 1),
  (2, 1),
  (4, 1),
  (3, 0),
  (3, 3),
  (2, 3),
  (1, 3),
  (1, 4),
  (2, 4),
  (3, 4),
  (2, 5)],
 [(0, 9), (1, 9), (2, 9), (1, 8), (0, 8), (0, 7), (0, 6), (1, 6), (0, 5)],
 [(2, 7), (3, 7), (4, 7), (4, 6), (3, 6), (4, 5), (4, 8), (3, 8), (4, 9)],
 [(0, 0), (1, 0), (0, 1)]]

In [13]:
tm.unexplored # should be blank after get_basins()

[]

In [14]:
# new spam since all basins already found
tm.get_basins()

[[(1, 2),
  (2, 2),
  (3, 2),
  (3, 1),
  (2, 1),
  (4, 1),
  (3, 0),
  (3, 3),
  (2, 3),
  (1, 3),
  (1, 4),
  (2, 4),
  (3, 4),
  (2, 5)],
 [(0, 9), (1, 9), (2, 9), (1, 8), (0, 8), (0, 7), (0, 6), (1, 6), (0, 5)],
 [(2, 7), (3, 7), (4, 7), (4, 6), (3, 6), (4, 5), (4, 8), (3, 8), (4, 9)],
 [(0, 0), (1, 0), (0, 1)]]

In [15]:
tm.get_biggest_basins()

[[(1, 2),
  (2, 2),
  (3, 2),
  (3, 1),
  (2, 1),
  (4, 1),
  (3, 0),
  (3, 3),
  (2, 3),
  (1, 3),
  (1, 4),
  (2, 4),
  (3, 4),
  (2, 5)]]

In [16]:
test_case = tm.get_biggest_basins(3)
sizes = [len(x) for x in test_case]
sizes[0]*sizes[1]*sizes[2]

1134

In [17]:
tm = TubeMap(data, verbose=False)
tm.map_all()

In [18]:
top3 = tm.get_biggest_basins(3)
sizes = [len(x) for x in top3 ]
sizes[0]*sizes[1]*sizes[2]

1050192